<a href="https://colab.research.google.com/github/rajeev-dw9/Scrapper_DSIP/blob/main/Scrapper_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Web Scrapping Assignment
# Name: Rajeev Ranjan Dwivedi
# Date: 26/10/2022
# Description: This program scrapes the data from the website (https://ascopubs.org/loi/jco) and stores it in a csv file.



In [10]:
# reset all variable, since it creates problem while running the code again and again
%reset -f 

In [11]:
# import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import csv


import time
import re
import os
import sys
import json
import datetime

In [12]:
# Get the data from the website
response = requests.get("https://ascopubs.org/loi/jco") # get the data from the website
soup = BeautifulSoup(response.text, "html.parser") # parse the data using beautiful soup

list_of_links = [] # list of links
all_links = soup.find_all("a") # find all links
for link in all_links:
    list_of_links.append(link.get("href")) # append the link to the list
    
# print(list_of_links) 

In [13]:
# Store all the links in a dataframe
df = pd.DataFrame(list_of_links, columns = ["Links"])
df.head(100)

,Links
0,#main
1,/journal/jco
2,https://signin.asco.org/oauth2/aus1goap5nQ33vo...
3,http://www.editorialmanager.com/jco-ascopubs/d...
4,https://apps.asco.org/EmailPreferences/Home/Up...
...,...
95,/toc/jco/40/7
96,/toc/jco/40/6
97,/toc/jco/40/5
98,/toc/jco/40/4


In [14]:
# remove columns with null values
df = df.dropna()
df.head(100)

# remove the links that are not related to the journal articles
df = df[df["Links"].str.contains("/toc/jco")]
df.shape

(967, 1)

In [15]:
from tqdm import tqdm

url = "https://ascopubs.org/toc/jco/"

data = []

for i in tqdm(range(1, 41)):
    out_url = url + str(i)
    index = 1
    
    year_data = []
    
    while True:
        titles = []
        authors = []
        articles = []
        
        target_url = out_url + "/" + str(index)
        
        response = requests.get(target_url) # get the data from the website
        if response.status_code == 404:
            break
        soup = BeautifulSoup(response.text, "html.parser")
        print(index, target_url, end=" ")
        
        tables = soup.find_all("table", {"class": "articleEntry"})
        for table in tables:
            # getting title
            title = table.find_all("span", {"class":"hlFld-Title"})
            if(len(title) > 0):
                titles.append(title[0].text)
            else:
                titles.append("")
                
            # getting author
            author = table.find_all("span", {"class":"entryAuthor"})
            if (len(author) > 0):
                authors.append(author[0].text)
            else:
                authors.append("")

            # getting pdf url
            pdf_url = table.find_all("a", {"class":"pdf"})[0].get('href')
            if not pdf_url.startswith('https://ascopubs.org/'):
                pdf_url = 'https://ascopubs.org' + pdf_url
            articles.append(pdf_url)
        year_data.append([titles, authors, articles])
        index += 1
        
    data.append(year_data)

  0%|          | 0/40 [00:00<?, ?it/s]

1 https://ascopubs.org/toc/jco/1/1 2 https://ascopubs.org/toc/jco/1/2 3 https://ascopubs.org/toc/jco/1/3 4 https://ascopubs.org/toc/jco/1/4 5 https://ascopubs.org/toc/jco/1/5 6 https://ascopubs.org/toc/jco/1/6 7 https://ascopubs.org/toc/jco/1/7 8 https://ascopubs.org/toc/jco/1/8 9 https://ascopubs.org/toc/jco/1/9 10 https://ascopubs.org/toc/jco/1/10 11 https://ascopubs.org/toc/jco/1/11 12 https://ascopubs.org/toc/jco/1/12 

  2%|▎         | 1/40 [00:13<09:02, 13.91s/it]

1 https://ascopubs.org/toc/jco/2/1 2 https://ascopubs.org/toc/jco/2/2 3 https://ascopubs.org/toc/jco/2/3 4 https://ascopubs.org/toc/jco/2/4 5 https://ascopubs.org/toc/jco/2/5 6 https://ascopubs.org/toc/jco/2/6 7 https://ascopubs.org/toc/jco/2/7 8 https://ascopubs.org/toc/jco/2/8 9 https://ascopubs.org/toc/jco/2/9 10 https://ascopubs.org/toc/jco/2/10 11 https://ascopubs.org/toc/jco/2/11 12 https://ascopubs.org/toc/jco/2/12 

  5%|▌         | 2/40 [00:32<10:35, 16.73s/it]

...

 95%|█████████▌| 38/40 [28:49<01:30, 45.40s/it]

1 https://ascopubs.org/toc/jco/39/1 2 https://ascopubs.org/toc/jco/39/2 3 https://ascopubs.org/toc/jco/39/3 4 https://ascopubs.org/toc/jco/39/4 5 https://ascopubs.org/toc/jco/39/5 6 https://ascopubs.org/toc/jco/39/6 7 https://ascopubs.org/toc/jco/39/7 8 https://ascopubs.org/toc/jco/39/8 9 https://ascopubs.org/toc/jco/39/9 10 https://ascopubs.org/toc/jco/39/10 11 https://ascopubs.org/toc/jco/39/11 12 https://ascopubs.org/toc/jco/39/12 13 https://ascopubs.org/toc/jco/39/13 14 https://ascopubs.org/toc/jco/39/14 15 https://ascopubs.org/toc/jco/39/15 16 https://ascopubs.org/toc/jco/39/16 17 https://ascopubs.org/toc/jco/39/17 18 https://ascopubs.org/toc/jco/39/18 19 https://ascopubs.org/toc/jco/39/19 20 https://ascopubs.org/toc/jco/39/20 21 https://ascopubs.org/toc/jco/39/21 22 https://ascopubs.org/toc/jco/39/22 23 https://ascopubs.org/toc/jco/39/23 24 https://ascopubs.org/toc/jco/39/24 25 https://ascopubs.org/toc/jco/39/25 26 https://ascopubs.org/toc/jco/39/26 27 https://ascopubs.org/toc/jc

 98%|█████████▊| 39/40 [29:36<00:45, 45.87s/it]

1 https://ascopubs.org/toc/jco/40/1 2 https://ascopubs.org/toc/jco/40/2 3 https://ascopubs.org/toc/jco/40/3 4 https://ascopubs.org/toc/jco/40/4 5 https://ascopubs.org/toc/jco/40/5 6 https://ascopubs.org/toc/jco/40/6 7 https://ascopubs.org/toc/jco/40/7 8 https://ascopubs.org/toc/jco/40/8 9 https://ascopubs.org/toc/jco/40/9 10 https://ascopubs.org/toc/jco/40/10 11 https://ascopubs.org/toc/jco/40/11 12 https://ascopubs.org/toc/jco/40/12 13 https://ascopubs.org/toc/jco/40/13 14 https://ascopubs.org/toc/jco/40/14 15 https://ascopubs.org/toc/jco/40/15 16 https://ascopubs.org/toc/jco/40/16 17 https://ascopubs.org/toc/jco/40/17 18 https://ascopubs.org/toc/jco/40/18 19 https://ascopubs.org/toc/jco/40/19 20 https://ascopubs.org/toc/jco/40/20 21 https://ascopubs.org/toc/jco/40/21 22 https://ascopubs.org/toc/jco/40/22 23 https://ascopubs.org/toc/jco/40/23 24 https://ascopubs.org/toc/jco/40/24 25 https://ascopubs.org/toc/jco/40/25 26 https://ascopubs.org/toc/jco/40/26 27 https://ascopubs.org/toc/jc

100%|██████████| 40/40 [30:07<00:00, 45.19s/it]


In [16]:
# creatting a single list of all type of data
all_titles = []
all_authors = []
all_articles = []
all_years = []
year = 1983
for item in data:
    for year_data in item:
        all_titles += year_data[0]
        all_authors += year_data[1]
        all_articles += year_data[2]
        all_years += [str(year) for x in range(len(year_data[0]))]
    year += 1

# creating a dictionary of data
data_dic = {'title':all_titles, 'author': all_authors, 'pdf_url':all_articles, 'year':all_years}

In [17]:
print(len(all_titles))
print(len(all_authors))
print(len(all_articles))
print(len(all_years))

27325
27325
27325
27325


In [18]:
df = pd.DataFrame(data_dic)
df.head(10)

,title,author,pdf_url,year
0,A Journal for Oncologists,,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
1,Adjuvant CMF in breast cancer: comparative 5-y...,Tancini et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
2,Histologic conversion in the non-Hodgkin's lym...,Acker et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
3,Feasibility study of combining metronidazole w...,Stewart et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
4,Constant infusion schedule for adriamycin: a p...,Lokich et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
5,Serum cyclophosphamide activity in patients tr...,Sarpel et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
6,Scalp hypothermia: a comparison of ice packs a...,Dean et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
7,Evaluation of prognostic factors in chemothera...,Eagan et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
8,The response of Ewing's sarcoma to sequential ...,Hayes et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
9,Methylglyoxal-bis(guanylhydrazone) (Methyl-GAG...,Warrell et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983


In [19]:
df.to_csv("data.csv", index=False)

In [20]:
# read csv file 
df_data = pd.read_csv("data.csv")
df_data.shape


(27325, 4)

In [21]:
df_data.head()

,title,author,pdf_url,year
0,A Journal for Oncologists,NaN,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
1,Adjuvant CMF in breast cancer: comparative 5-y...,Tancini et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
2,Histologic conversion in the non-Hodgkin's lym...,Acker et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
3,Feasibility study of combining metronidazole w...,Stewart et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
4,Constant infusion schedule for adriamycin: a p...,Lokich et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983


In [22]:
# remove columns with NaN values 
df_data = df_data.dropna()
df_data.head()

,title,author,pdf_url,year
1,Adjuvant CMF in breast cancer: comparative 5-y...,Tancini et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
2,Histologic conversion in the non-Hodgkin's lym...,Acker et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
3,Feasibility study of combining metronidazole w...,Stewart et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
4,Constant infusion schedule for adriamycin: a p...,Lokich et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
5,Serum cyclophosphamide activity in patients tr...,Sarpel et al.,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983


In [23]:
# remove the rows with empty title
df_data = df_data[df_data["title"] != ""]
df_data.head()
df_data.shape

# remove the rows with empty author
df_data = df_data[df_data["author"] != ""]
df_data.shape

# remove the rows with empty pdf_url
df_data = df_data[df_data["pdf_url"] != ""]
df_data.shape



(26245, 4)

In [24]:
# remove "et al." from the author column
df_data["author"] = df_data["author"].str.replace("et al.", "")
df_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,title,author,pdf_url,year
1,Adjuvant CMF in breast cancer: comparative 5-y...,Tancini,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
2,Histologic conversion in the non-Hodgkin's lym...,Acker,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
3,Feasibility study of combining metronidazole w...,Stewart,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
4,Constant infusion schedule for adriamycin: a p...,Lokich,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983
5,Serum cyclophosphamide activity in patients tr...,Sarpel,https://ascopubs.org/doi/pdf/10.1200/JCO.1983....,1983


In [25]:
# find the unique authors
unique_authors = df_data["author"].unique()
print(len(unique_authors)  )

# find the unique titles
unique_titles = df_data["title"].unique()
print(len(unique_titles))

# find the unique pdf_urls
unique_pdf_urls = df_data["pdf_url"].unique()
print(len(unique_pdf_urls))

# find the unique years
unique_years = df_data["year"].unique()
print(len(unique_years))



11493
25051
26114
40


In [26]:
# make a column for the title
df_data["title"] = df_data["title"].str.lower()
df_data.head()
# store tilte col in a list
title_list = df_data["title"].tolist()
title_list



['adjuvant cmf in breast cancer: comparative 5-year results of 12 versus 6 cycles.',
 "histologic conversion in the non-hodgkin's lymphomas.",
 'feasibility study of combining metronidazole with chemotherapy.',
 'constant infusion schedule for adriamycin: a phase i-ii clinical trial of a 30-day schedule by ambulatory pump delivery system.',
 'serum cyclophosphamide activity in patients treated for small cell carcinoma of the lung.',
 'scalp hypothermia: a comparison of ice packs and the kold kap in the prevention of doxorubicin-induced alopecia.',
 'evaluation of prognostic factors in chemotherapy of recurrent brain tumors.',
 "the response of ewing's sarcoma to sequential cyclophosphamide and adriamycin induction therapy.",
 'methylglyoxal-bis(guanylhydrazone) (methyl-gag): current status and future prospects.',
 'reflections on medical oncology: an appeal for better clinical trials and improved reporting of their results.',
 'karnofsky memorial lecture. breaking the cure barrier.',
 